In [8]:
from functools import partial
import numpy as np
import tensorflow as tf
import jax
from jax.config import config

config.update("jax_enable_x64", True)
from jax import numpy as jnp
from jax.experimental import optimizers
import tensorcircuit as tc

zz = np.kron(tc.gates._z_matrix, tc.gates._z_matrix)

l1 = 100
p1 = [0 for i in range(l1)]
p2 = [0 for i in range(l1)]
h = [1 for i in range(l1)]


def rzz(c,i,j):
    c.cnot(i,i+1)
    c.rz(i+1,p2[j])
    c.cnot(i,i+1)
    return c

def energy(c: tc.Circuit):
    e = 0.0
    n = c._nqubits
    for i in range(n):
        e += h[i] * c.expectation((tc.gates.x(), [i]))  # <X_i>
    for i in range(n - 1):  # OBC
        e += (-1.0) * c.expectation(
            (tc.gates.z(), [i]), (tc.gates.z(), [(i + 1) % n])
        )  # <Z_iZ_{i+1}>
    return tc.backend.real(e)
    

def ex(c):
    n=c._nqubits
    k=100
    for i in range(n):
        c.h(i)
    for j in range(k):
        for i in range(n):
            c.rx(i,p1[j])
        for i in range(n-1):
            c=rzz(c,i,j)
    return energy(c)
    
    